In [2]:
import pandas as pd
import json
import ast

In [3]:
# Load data from CSV
data = pd.read_csv('./dataset/training.csv')

# Load keywords and questions from JSON files
with open('./dataset/diagnosis_keywords.json', 'r') as keywords_file:
    keywords_data = json.load(keywords_file)

with open('./dataset/question_keywords.json', 'r') as questions_file:
    questions_data = json.load(questions_file)

print("DATASETS LOADED SUCCESSFULLY")

DATASETS LOADED SUCCESSFULLY


In [4]:
# Create a mapping of keywords to questions
keyword_to_question = {item: questions_data[item]['question_en'] for item in questions_data}

print("KEYWORDS MAPPED TO QUESTIONS SUCCESSFULLY")

# Create a mapping of keywords to diagnoses
keyword_to_diagnosis = {item: keywords_data[item]['cond-name-eng'] for item in keywords_data}

print("KEYWORDS MAPPED TO DIAGNOSES SUCCESSFULLY")

KEYWORDS MAPPED TO QUESTIONS SUCCESSFULLY
KEYWORDS MAPPED TO DIAGNOSES SUCCESSFULLY


In [12]:
# Initialize lists to store feature vectors and labels
feature_vectors = []
labels = []

# Process each row in the CSV file
for index, row in data.iterrows():
    # Extract values from the row
    age = row['AGE']
    sex = 1 if row['SEX'] == 'F' else 0
    initial_evidence = row['INITIAL_EVIDENCE']
    differential_diagnosis = ast.literal_eval(row['DIFFERENTIAL_DIAGNOSIS'])
 
    # Create a binary vector for evidence questions
    evidence_vector = [1 if keyword in initial_evidence else 0 for keyword in keyword_to_question.keys()]

    # Create a list of likelihood percentages
    percentages = [0.0] * len(keyword_to_diagnosis)

    for element in differential_diagnosis:
        diagnosis = element[0]
        likelihood = element[1]
        if diagnosis in keyword_to_diagnosis:
            percentages[list(keyword_to_diagnosis.keys()).index(diagnosis)] = likelihood

    # Create the feature vector
    feature_vector = [age, sex] 
    # + evidence_vector + percentages
    feature_vector.append(evidence_vector)
    feature_vector.append(percentages)

    # Append the feature vector and label (e.g., pathology) to the lists
    feature_vectors.append(feature_vector)
    labels.append(keywords_data[row['PATHOLOGY']]["cond-name-eng"])


In [14]:
print(feature_vectors[0])

[18, 0, [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.07333003867293564, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.12429044460990353, 0.0, 0.04984197229703562, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11367177304035844, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19171203430383882, 0.0, 0.0, 0.1607809719801254, 0.11057936110639896, 0.175793403989

In [16]:
# Convert lists to a DataFrame for further processing or modeling
column_names = ['AGE', 'SEX', 'EVIDENCE_VECTOR', 'PERCENTAGES']
feature_df = pd.DataFrame(feature_vectors, columns=column_names)
label_df = pd.DataFrame(labels, columns=['PATHOLOGY'])

In [18]:
# Print or save the feature DataFrame for modeling
print(label_df)

                       PATHOLOGY
0                           URTI
1        HIV (initial infection)
2                      Pneumonia
3                           URTI
4                           URTI
...                          ...
1025597             Epiglottitis
1025598             Epiglottitis
1025599             Epiglottitis
1025600             Epiglottitis
1025601             Epiglottitis

[1025602 rows x 1 columns]


In [19]:
# Save feature_df to a CSV file
feature_df.to_csv('./dataset/feature_data.csv', index=False)

# Save label_df to a CSV file
label_df.to_csv('./dataset/label_data.csv', index=False)

print("DataFrames exported to CSV files successfully.")

DataFrames exported to CSV files successfully.
